In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
import scipy
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler

#on transforme nos datasets en dataframes

true = pd.read_csv("C:/Users/admin/Desktop/machine_learning_articles-main/DATA/True.csv")
fake = pd.read_csv("C:/Users/admin/Desktop/machine_learning_articles-main/DATA/Fake.csv")

true['class'] = 1
fake['class'] = 0
fake['title'] = fake['title'].str.lower()
fake['text'] = fake['text'].str.lower()
true['title'] = true['title'].str.lower()
true['text'] = true['text'].str.lower()
fake['title'] = fake['title'].str.replace('[^\w\s]', '')
fake['text'] = fake['text'].str.replace('[^\w\s]', '')
true['title'] = true['title'].str.replace('[^\w\s]', '')
true['text'] = true['text'].str.replace('[^\w\s]', '')
true['combined_text'] = true['title'] + ' ' + true['text']
fake['combined_text'] = fake['title'] + ' ' + fake['text']
dataset=pd.concat([true, fake])

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
# Function to remove numbers from a string
def remove_numbers(text):
    return re.sub(r'\d+', '', text)
# Fonction pour garder uniquement les lettres et les espaces
def keep_letters_and_spaces(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

dataset['final_text'] = dataset['combined_text'].apply(remove_stop_words)
dataset['final_text2'] = dataset['final_text'].apply(remove_numbers)
dataset['final_text3'] = dataset['final_text2'].apply(keep_letters_and_spaces)
df = dataset[['final_text3','class']].copy()


max_features_value=1000
tfidf = TfidfVectorizer(max_features=max_features_value)
tfidf.fit(df['final_text3'])
tfidf_features = tfidf.transform(df['final_text3'])

tfidf_features_sparse = csr_matrix(tfidf_features)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_features_sparse, columns=tfidf.get_feature_names_out())

column_name = 'class'
tfidf_df[column_name] = df[column_name].values

df['len']=df['final_text3'].str.len()
df['num']=df['final_text3'].str.split().map(lambda x: len(x))

scaler = MinMaxScaler()

column_name = 'len'
column_name2 = 'num'

df[column_name] = scaler.fit_transform(df[[column_name]])
df[column_name2] = scaler.fit_transform(df[[column_name2]])


tfidf_df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,able,absolutely,access,according,account,accused,across,act,action,actions,...,wrong,wrote,year,yearold,years,yet,york,you,young,class
0,0,0,0,0,0,0,0,0,0.038091,0,...,0,0,0,0,0.078363,0.035106,0,0,0.041585,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0.035544,0,0,0,0.092775,0,0,1
3,0,0,0,0.048577,0,0.065308,0,0,0,0.072664,...,0,0,0.043781,0,0,0,0.171412,0,0,1
4,0,0,0,0.128131,0,0,0,0,0,0,...,0,0.034531,0.11548,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,0.055766,0,0,0,0,0,0,0.107019,0,0.126243,...,0,0,0,0,0,0,0.049634,0,0,0
44894,0,0,0.096987,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44895,0,0,0,0.015172,0,0.010199,0.009699,0.00962,0,0,...,0,0,0.006837,0,0.041381,0.009269,0.017846,0,0,0
44896,0,0,0,0,0,0,0,0,0,0,...,0,0,0.036049,0,0,0,0,0,0,0


In [12]:
#Modèle 1 : Random Forest
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# On sépare les données en caractéristiques et classe
X = tfidf_df.drop('class', axis=1)
y = tfidf_df['class']

# On définit les mesures de performance MAE, RMSE, MAPE et R^2
def evaluation_performance_RF(y_true, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return accuracy, mae, rmse, r2

# On définit les paramètres du Random Forest
n_estimators = 100
max_depth = None

# On définit le nombre de folds pour la cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

accuracy_scores_RF = []
mae_scores_RF = []
rmse_scores_RF = []
r2_scores_RF = []

# On lance la cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # On créé et on ajuste le modèle Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    rf.fit(X_train, y_train)

    # On lance la prédiction sur l'ensemble de test
    y_pred = rf.predict(X_test)

    # On procède à l'évaluation des performances
    accuracy, mae, rmse, r2 = evaluation_performance_RF(y_test, y_pred)

    accuracy_scores_RF.append(accuracy)
    mae_scores_RF.append(mae)
    rmse_scores_RF.append(rmse)
    r2_scores_RF.append(r2)


# On calcule la moyenne des scores de performance
moy_accuracy_RF = np.mean(accuracy_scores_RF)
moy_mae_RF = np.mean(mae_scores_RF)
moy_rmse_RF = np.mean(rmse_scores_RF)
moy_r2_RF = np.mean(r2_scores_RF)

print("Random Forest : ")
print("Accuracy : ", moy_accuracy_RF)
print("MAE : ", moy_mae_RF)
print("RMSE : ", moy_rmse_RF)
print("R² : ", moy_r2_RF)


Random Forest : 
Accuracy :  0.9977950059666567
MAE :  0.0022049940333433295
RMSE :  0.04651349699000451
R² :  0.9911604653760158


In [13]:
#Modèle 2 : Decision tree
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# On sépare les données en caractéristiques et classe
X = tfidf_df.drop('class', axis=1)
y = tfidf_df['class']

# On définit les mesures de performance MAE, RMSE, MAPE et R^2
def evaluation_performance_DT(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return accuracy, mae, rmse, r2

# On définit les paramètres de l'arbre de décision
max_depth = None

# On définit le nombre de folds pour la cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

accuracy_scores_DT = []
mae_scores_DT = []
rmse_scores_DT = []
r2_scores_DT = []

#On lance la cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # On créé et on ajuste le modèle d'arbre de décision
    dt = DecisionTreeClassifier(max_depth=max_depth)
    dt.fit(X_train, y_train)

    # On lance la prédiction sur l'ensemble de test
    y_pred = dt.predict(X_test)

    # On procéde à l'évaluation des performances
    accuracy, mae, rmse, r2 = evaluation_performance_DT(y_test, y_pred)

    accuracy_scores_DT.append(accuracy)
    mae_scores_DT.append(mae)
    rmse_scores_DT.append(rmse)
    r2_scores_DT.append(r2)

# On calcule la moyenne des scores de performance
avg_accuracy_DT = np.mean(accuracy_scores_DT)
avg_mae_DT = np.mean(mae_scores_DT)
avg_rmse_DT = np.mean(rmse_scores_DT)
avg_r2_DT = np.mean(r2_scores_DT)

print("Arbre de décision :")
print("Accuracy : ", avg_accuracy_DT)
print("MAE : ", avg_mae_DT)
print("RMSE : ", avg_rmse_DT)
print("R² : ", avg_r2_DT)


Arbre de décision :
Accuracy :  0.9953672600581758
MAE :  0.004632739941824168
RMSE :  0.06799750696528775
R² :  0.9814282721118441


In [9]:
from joblib import dump, load

# Sauvegarder le modèle
dump(tfidf_df, 'tdf.joblib')
print("Modèle sauvegardé sous 'decision_tree_model.joblib'.")

# Charger le modèle
loaded_model = load('tdf.joblib')
print("Modèle chargé avec succès.")


Modèle sauvegardé sous 'decision_tree_model.joblib'.
Modèle chargé avec succès.


In [14]:
#Modèle 3 : SVM
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# On sépare les données en caractéristiques et classe
X = tfidf_df.drop('class', axis=1)
y = tfidf_df['class']

# On définit les mesures de performance MAE, RMSE, MAPE et R^2
def evaluate_performance_SVM(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return accuracy, mae, rmse, r2

# On définit les paramètres de SVM
C = 1.0
kernel = 'rbf'

# On définit le nombre de folds pour la cross-validation
n_splits = 2
kf = KFold(n_splits=n_splits, shuffle=True)

accuracy_scores_SVM = []
mae_scores_SVM = []
rmse_scores_SVM = []
r2_scores_SVM = []

# On lance la boucle de cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # On créé et ajusteme le modèle SVM
    svm = SVC(C=C, kernel=kernel)
    svm.fit(X_train.values, y_train.values)  # Utilisation de .values pour convertir en tableaux NumPy

    # On lance la prédiction sur l'ensemble de test
    y_pred = svm.predict(X_test.values)  # Utilisation de .values pour convertir en tableaux NumPy

    # On lance l'évaluation des performances
    accuracy, mae, rmse, r2 = evaluate_performance_SVM(y_test.values, y_pred)  # Utilisation de .values pour convertir en tableaux NumPy

    accuracy_scores_SVM.append(accuracy)
    mae_scores_SVM.append(mae)
    rmse_scores_SVM.append(rmse)
    r2_scores_SVM.append(r2)

# On calcule la moyenne des scores de performance
avg_accuracy_SVM = np.mean(accuracy_scores_SVM)
avg_mae_SVM = np.mean(mae_scores_SVM)
avg_rmse_SVM = np.mean(rmse_scores_SVM)
avg_r2_SVM = np.mean(r2_scores_SVM)

print("SVM :")
print("Accuracy : ", avg_accuracy_SVM)
print("MAE : ", avg_mae_SVM)
print("RMSE : ", avg_rmse_SVM)
print("R² : ", avg_r2_SVM)


SVM :
Accuracy :  0.9936077330838791
MAE :  0.006392266916120985
RMSE :  0.07983462627922196
R² :  0.9743752813961783


In [7]:
#Modèle 4 : LSTM
from tensorflow.keras.preprocessing import text
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler

# On sépare les données en caractéristiques et classe
X = tfidf_df.drop('class', axis=1)
y = tfidf_df['class']

# On définit des paramètres du modèle
vocab_size = max_features_value
embedding_dim = 40
max_length = 100

# On convertit les phrases en séquences d'entiers
tokenizer = text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df['final_text3'])
sequences = tokenizer.texts_to_sequences(df['final_text3'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# On procède à la division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# On créé le modèle LSTM
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# On lance la compilation du modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# On lance l'entraînement du modèle
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)

# On fait les prédictions sur les données de test
y_pred = model.predict(X_test)

# On calcule les mesures de performance
accuracy = accuracy_score(y_test, np.round(y_pred))
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("LSTM")
print("Accuracy:", accuracy)
print("MAE: ", mae)
print("RMSE: ", rmse)
print("R²: ", r2)

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1123/1123 - 153s - 137ms/step - accuracy: 0.9909 - loss: 0.0328 - val_accuracy: 0.9962 - val_loss: 0.0173
Epoch 2/10
1123/1123 - 127s - 113ms/step - accuracy: 0.9985 - loss: 0.0067 - val_accuracy: 0.9974 - val_loss: 0.0136
Epoch 3/10
1123/1123 - 132s - 117ms/step - accuracy: 0.9990 - loss: 0.0050 - val_accuracy: 0.9960 - val_loss: 0.0181
Epoch 4/10
1123/1123 - 128s - 114ms/step - accuracy: 0.9991 - loss: 0.0037 - val_accuracy: 0.9976 - val_loss: 0.0158
Epoch 5/10
1123/1123 - 141s - 125ms/step - accuracy: 0.9993 - loss: 0.0031 - val_accuracy: 0.9974 - val_loss: 0.0143
Epoch 6/10
1123/1123 - 138s - 123ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 0.9971 - val_loss: 0.0154
Epoch 7/10
1123/1123 - 197s - 176ms/step - accuracy: 0.9996 - loss: 0.0018 - val_accuracy: 0.9971 - val_loss: 0.0166
Epoch 8/10
1123/1123 - 128s - 114ms/step - accuracy: 0.9998 - loss: 8.6731e-04 - val_accuracy: 0.9978 - val_loss: 0.0180
Epoch 9/10
1123/1123 - 125s - 111ms/step - accuracy: 0.9996 

In [3]:
pip install keras.preprocessing


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
#Modèle 5 : CNN
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D, Dropout
from sklearn.model_selection import train_test_split

# Assuming you have a Pandas DataFrame called 'df' with 1001 columns

shuffled_df = tfidf_df.sample(frac=1).reset_index(drop=True)

# Convert DataFrame to numpy array
data = shuffled_df.to_numpy()

# Split the data into input features (X) and target variable (y)
X = data[:, :1000]  # Assuming the first 1000 columns are input features
y = data[:, 1000]  # Assuming the last column is the target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define other necessary variables
embedding_dim = 100
num_filters = 128
dropout_rate = 0.2

# Create the model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=embedding_dim, input_length=1000))  # Assuming vocabulary size is 10000
model.add(Conv1D(filters=num_filters, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
print(model.summary())
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)
# Évaluez le modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy:", accuracy)

# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
# On calcule les mesures de performance
accuracy = accuracy_score(y_test, np.round(y_pred))
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("CNN")
print("Accuracy:", accuracy)
print("MAE: ", mae)
print("RMSE: ", rmse)
print("R²: ", r2)

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
1123/1123 - 239s - 213ms/step - accuracy: 0.5210 - loss: 0.6929 - val_accuracy: 0.5182 - val_loss: 0.6947
Epoch 2/10
1123/1123 - 221s - 197ms/step - accuracy: 0.5237 - loss: 0.6920 - val_accuracy: 0.5182 - val_loss: 0.6922
Epoch 3/10
1123/1123 - 221s - 196ms/step - accuracy: 0.5242 - loss: 0.6917 - val_accuracy: 0.5182 - val_loss: 0.6921
Epoch 4/10
1123/1123 - 222s - 197ms/step - accuracy: 0.5242 - loss: 0.6918 - val_accuracy: 0.5182 - val_loss: 0.6922
Epoch 5/10
1123/1123 - 269s - 239ms/step - accuracy: 0.5244 - loss: 0.6920 - val_accuracy: 0.5182 - val_loss: 0.6921
Epoch 6/10
1123/1123 - 255s - 227ms/step - accuracy: 0.5240 - loss: 0.6919 - val_accuracy: 0.5182 - val_loss: 0.6921
Epoch 7/10
1123/1123 - 238s - 212ms/step - accuracy: 0.5238 - loss: 0.6919 - val_accuracy: 0.5182 - val_loss: 0.6921
Epoch 8/10
1123/1123 - 229s - 204ms/step - accuracy: 0.5242 - loss: 0.6918 - val_accuracy: 0.5182 - val_loss: 0.6922
Epoch 9/10
1123/1123 - 228s - 203ms/step - accuracy: 0.5242